# 01. Time decompression relay
TDR, but with the key parts in a separate script for convenience.

This version uses the relayed fO2 of the EQ3 fluid rather than the fO2 set by the solid.

In [2]:
with open('TDR_preamble_fO2.py') as file:  # THIS IS THE RELAYED FO2
    exec(file.read())

columnDF = pd.read_csv('columns/output_75km_0p1inv.csv')

ImportError: Building module dew2019 failed: ["ImportError: dlopen(/Users/sm905/.pyxbld/lib.macosx-11.0-arm64-cpython-39/dew2019.cpython-39-darwin.so, 0x0002): Library not loaded: ./src/libswimdew.dylib\n  Referenced from: <A9D450D0-EC48-3B45-8FDF-434C6699B9A8> /Users/sm905/.pyxbld/lib.macosx-11.0-arm64-cpython-39/dew2019.cpython-39-darwin.so\n  Reason: tried: './src/libswimdew.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS./src/libswimdew.dylib' (no such file), './src/libswimdew.dylib' (no such file)\n"]

## A. Create our solid

In [2]:
# List of reactants formed from mineral dictionaries
olvDict = {'reactant': 'OLIVINE(SS)',
           'moles': 5.9,
           'composition': {'FORSTERITE': 0.9, 'FAYALITE': 0.1}}

opxDict = {'reactant': 'ORTHOPYROXENE(SS)',
           'moles': 1.9,
           'composition': {'ENSTATITE_OR': 0.9, 'FERROSILITE': 0.1}}

cpxDict = {'reactant': 'CLINOPYROXENE(SS)',
           'moles': 0.2,
           'composition': {'DIOPSIDE': 0.9, 'HEDENBERGITE': 0.1}}

spnDict = {'reactant': 'SPINEL',
           'moles': 0.04}

mgtDict = {'reactant': 'MAGNETITE',
           'moles': 0.04}

startingReactants = [olvDict, opxDict, cpxDict, spnDict, mgtDict]

# Quickly check that our starting fluid-rock ratio (F/R) is going to be 1 (or whatever it is)
startingReactants = pDF.reactantToFR(startingReactants, 1.0, dew_system)

# Assign pressures and temperatures

pressures = columnDF['Pressure (kbar)'].tolist()  # in kbar
pressures = [i*1000 for i in pressures]  # in bar

temperatures = columnDF['Temperature (degC)'].tolist()  # in degC
temperatures = [i+273.15 for i in temperatures]  # in K

logfO2s = columnDF['log fO2'].tolist()

# Get starting PT for fluid interpolation
sourcePressure = pressures[0]
sourceTemperature = temperatures[0]

# Create our preliminary compositional stack
startingComposition = pDF.reactantToDF(startingReactants, dew_system)
startingComposition = pd.concat([startingComposition]*len(pressures))
startingComposition['Pressure (bars)'] = pressures
startingComposition['Temperature (K)'] = temperatures
# startingComposition['log fO2'] = logfO2s
startingComposition['Time'] = [0]*len(pressures)

print(startingComposition)


    FORSTERITE  FAYALITE  ENSTATITE_OR  FERROSILITE  DIOPSIDE(CPX)  \
0     4.728685  0.525409      1.522797       0.1692       0.160294   
0     4.728685  0.525409      1.522797       0.1692       0.160294   
0     4.728685  0.525409      1.522797       0.1692       0.160294   
0     4.728685  0.525409      1.522797       0.1692       0.160294   
0     4.728685  0.525409      1.522797       0.1692       0.160294   
..         ...       ...           ...          ...            ...   
0     4.728685  0.525409      1.522797       0.1692       0.160294   
0     4.728685  0.525409      1.522797       0.1692       0.160294   
0     4.728685  0.525409      1.522797       0.1692       0.160294   
0     4.728685  0.525409      1.522797       0.1692       0.160294   
0     4.728685  0.525409      1.522797       0.1692       0.160294   

    HEDENBERGITE(CPX)    SPINEL  MAGNETITE  Pressure (bars)  Temperature (K)  \
0             0.01781  0.035621   0.035621          20200.0       700.518422   

## B. Create our fluid

In [3]:
startingFluid = createHybridFluid([1, 0, 0], sourceTemperature, sourcePressure)

## C. Run our column

In [4]:
from datetime import date
today = date.today()
todayStr = str(today)
todayStr.replace('-','')

'20240501'

In [5]:
startingFluid = createHybridFluid([1, 0, 0], sourceTemperature, sourcePressure)
decompressionRelay(startingComposition, startingFluid[0], startingFluid[1],
                   dew_system, timesteps=5, startingFRRatio=10.0, terminalFRRatio=0.1,
                   exportPath='columns_data/'+todayStr+'_output_75km_0p1inv_DMMfluid_t5_FR10_p04mgt', FRRatioOption0='fixedStart')

Time loop started


/opt/conda/lib/python3.9/site-packages/pyDEW/eq6_output.py:46: UserWarning: The pH output could not be read.
  warn("The pH output could not be read.")


No mineral product found.
Calculation at [n=143, t=0] has failed: using fluid/solid results from previous n/t


/opt/conda/lib/python3.9/site-packages/pyDEW/eq6_output.py:46: UserWarning: The pH output could not be read.
  warn("The pH output could not be read.")


No mineral product found.
Calculation at [n=145, t=0] has failed: using fluid/solid results from previous n/t


...

No mineral product found.


/opt/conda/lib/python3.9/site-packages/pyDEW/eq6_output.py:46: UserWarning: The pH output could not be read.
  warn("The pH output could not be read.")


No mineral product found.


/opt/conda/lib/python3.9/site-packages/pyDEW/eq6_output.py:46: UserWarning: The pH output could not be read.
  warn("The pH output could not be read.")


No mineral product found.
Timestep t=0 is complete, with 148 total iterations performed
Calculations : 144
Passes (F/R) : 0
Passes (error) : 5
No mineral product found.
Solid solution disabled.
Mineral output at [n=38, t=1] is not a pd.DataFrame, raising Error
Calculation at [n=38, t=1] has failed: using fluid/solid results from previous n/t
No mineral product found.
Solid solution disabled.
Mineral output at [n=42, t=1] is not a pd.DataFrame, raising Error
Calculation at [n=42, t=1] has failed: using fluid/solid results from previous n/t
Timestep t=1 is complete, with 296 total iterations performed
Calculations : 147
Passes (F/R) : 0
Passes (error) : 2
-27.3
[-47.0069 -47.0069 -47.0069 -47.0069 -47.0069 -47.0069 -47.0069 -47.0069
 -47.0069 -47.0069 -47.0069 -47.0069 -47.0069 -47.0069 -47.0069 -47.0069
 -47.0069 -47.0069 -47.0069 -47.0069 -47.0069 -47.0069 -47.0069 -47.0069
 -47.0069 -47.0069 -47.0069 -47.0069 -47.0069 -47.0069 -47.0069 -47.0069
 -47.0069 -47.0069 -47.0069 -47.0069 -47